In [1]:
from swan_vis import *

## Creating tss / tes / ic adatas from cerberus

In [36]:
def create_cerberus_adata(sg, mode):
    """
    Parameters:
        mode (str): {'ic', 'tss', 'tes'}
    """
    id_col = "{}_id".format(mode)
    name_col = "{}_name".format(mode)

    # merge ic, tss, tes info with transcript counts
    gb_cols = ["gid", "gname", id_col]
    subset_cols = copy.deepcopy(gb_cols)
    if mode in ["tss", "tes"]:
        end = True
        vert_col = "{}_vertex".format(mode)
        subset_cols.append("loc_path")
        if mode == "tss":
            ind = 0
        elif mode == "tes":
            ind = -1
    else:
        end = False

    df = self.t_df[subset_cols].reset_index()

    # merge with vertex id
    if end:
        df[vert_col] = [path[ind] for path in df.loc_path.values.tolist()]
        df.drop("loc_path", axis=1, inplace=True)
        gb_cols.append(vert_col)

    t_counts = self.get_transcript_abundance(kind="counts")
    t_counts = t_counts.merge(df, how="left", on="tid")
    # if mode == 'tes':
    #     print(t_counts.loc[t_counts.tes_id == 'ENCODEHG000058837_1', 'hl60_m2_72hr_1_1'])

    # gb and sum counts over the different features
    t_counts.drop("tid", axis=1, inplace=True)
    # print(mode)
    # print(gb_cols)
    t_counts = t_counts.groupby(gb_cols).sum()

    # get separate components of the table
    X = sparse.csr_matrix(t_counts.transpose().values)
    obs = self.adata.obs

    # add name of thing
    t_counts.reset_index(inplace=True)
    t_counts[name_col] = (
        t_counts.gname + "_" + t_counts[id_col].str.split("_", expand=True)[1]
    )
    gb_cols.append(name_col)
    var = t_counts[gb_cols]
    var.set_index(id_col, inplace=True)

    if end:
        var.rename({vert_col: "vertex"}, axis=1, inplace=True)

    return obs, var, X


def create_feat_adata(self, kind):
    """
    Create a tss / tes / ic-level adata object. Enables calculating tss / tes
    usage across samples.

    Parameters:
        kind (str): {'tss', 'ic', 'tes'}
    """

    # check to see if end information is already present in swangraph
    id_col = "{}_id".format(kind)
    if id_col in self.t_df.columns:
        obs, var, X = create_cerberus_adata(sg, kind)

    else:
        # limit to only expresed transcripts
        t_df = self.t_df.copy(deep=True)
        t_df = t_df.loc[self.adata.var.index.tolist()]
        df = get_ends(t_df, kind)

        # get a mergeable transcript expression df
        tid = self.adata.var.index.tolist()
        obs = self.adata.obs.index.tolist()
        data = self.adata.layers["counts"].transpose()
        t_exp_df = pd.DataFrame.sparse.from_spmatrix(columns=obs, data=data, index=tid)
        t_exp_df = t_exp_df.merge(t_df, how="left", left_index=True, right_index=True)

        # merge counts per transcript with end expression
        df = df.merge(t_exp_df, how="left", left_index=True, right_index=True)

        # sort based on vertex id
        df.sort_index(inplace=True, ascending=True)

        # set index to gene ID, gene name, and vertex id
        df.reset_index(drop=True, inplace=True)
        df.set_index(["gid", "gname", "vertex_id"], inplace=True)
        df = df[self.datasets]

        # groupby on gene and assign each unique TSS / gene combo an ID
        # use dense representation b/c I've already removed 0 counts and sparse
        # gb operations are known to be slow in pandas
        # https://github.com/pandas-dev/pandas/issues/36123
        # maybe try this? :
        # https://cmdlinetips.com/2019/03/how-to-write-pandas-groupby-function-using-sparse-matrix/
        id_col = "{}_id".format(kind)
        name_col = "{}_name".format(kind)
        df = df.copy()
        df.reset_index(inplace=True)
        df[self.datasets] = df[self.datasets].sparse.to_dense()
        df = df.groupby(["gid", "gname", "vertex_id"]).sum().reset_index()

        df["end_gene_num"] = (
            df.sort_values(["gid", "vertex_id"], ascending=[True, True])
            .groupby(["gid"])
            .cumcount()
            + 1
        )
        df[id_col] = df["gid"] + "_" + df["end_gene_num"].astype(str)
        df[name_col] = df["gname"] + "_" + df["end_gene_num"].astype(str)
        df.drop("end_gene_num", axis=1, inplace=True)

        # obs, var, and X tables for new data
        var_cols = ["gid", "gname", "vertex_id", id_col, name_col]
        var = df[var_cols]
        var.set_index("{}_id".format(kind), inplace=True)
        df.drop(var_cols, axis=1, inplace=True)
        df = df[self.adata.obs.index.tolist()]
        X = sparse.csr_matrix(df.transpose().values)
        obs = self.adata.obs

    # create anndata
    adata = anndata.AnnData(var=var, obs=obs, X=X)

    # add counts and tpm as layers
    adata.layers["counts"] = adata.X
    adata.layers["tpm"] = sparse.csr_matrix(calc_tpm(adata, recalc=True).to_numpy())
    if not self.sc:
        adata.layers["pi"] = sparse.csr_matrix(calc_pi(adata, adata.var)[0].to_numpy())

    # assign adata and clean up unstructured data if needed
    if kind == "tss":
        if self.has_abundance():
            adata.uns = self.tss_adata.uns
        self.tss_adata = adata

    elif kind == "tes":
        if self.has_abundance():
            adata.uns = self.tss_adata.uns
        self.tes_adata = adata

    elif kind == "ic":
        if self.has_abundance():
            adata.uns = self.ic_adata.uns
        self.ic_adata = adata

In [39]:
fname = "/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/lr_bulk/cerberus/swan.p"
sg = read(fname)
sg.ic_adata = anndata.AnnData()

Read in graph from /Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/lr_bulk/cerberus/swan.p


In [40]:
create_feat_adata(sg, kind="tss")
create_feat_adata(sg, kind="tes")
create_feat_adata(sg, kind="ic")

/Users/fairliereese/Documents/programming/mortazavi_lab/bin/swan_vis/swan_vis/utils.py:410: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df.reset_index(inplace=True)
/Users/fairliereese/Documents/programming/mortazavi_lab/bin/swan_vis/swan_vis/utils.py:410: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df.reset_index(inplace=True)
/Users/fairliereese/Documents/programming/mortazavi_lab/bin/swan_vis/swan_vis/utils.py:410: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` 

In [46]:
sg.tss_adata

AnnData object with n_obs × n_vars = 138 × 71958
    obs: 'dataset', 'total_counts', 'classification', 'sample', 'health_status'
    var: 'gid', 'gname', 'vertex', 'tss_name'
    uns: 'sample_colors', '{}_dict', 'health_status_colors'
    layers: 'counts', 'tpm', 'pi'

In [45]:
sg.tes_adata

AnnData object with n_obs × n_vars = 138 × 86085
    obs: 'dataset', 'total_counts', 'classification', 'sample', 'health_status'
    var: 'gid', 'gname', 'vertex', 'tes_name'
    uns: 'sample_colors', '{}_dict', 'health_status_colors'
    layers: 'counts', 'tpm', 'pi'

In [44]:
sg.ic_adata

AnnData object with n_obs × n_vars = 138 × 132640
    obs: 'dataset', 'total_counts', 'classification', 'sample', 'health_status'
    var: 'gid', 'gname', 'ic_name'
    layers: 'counts', 'tpm', 'pi'

## Adding tss / tes / ic from cerberus 

In [2]:
def abundance_to_adata(sg, counts_file, how="iso"):
    # read in abundance file
    check_file_loc(counts_file, "abundance matrix")
    try:
        df = pd.read_csv(counts_file, sep="\t")
    except:
        raise ValueError("Problem reading expression matrix {}".format(counts_file))

    # check if abundance matrix is a talon abundance matrix
    cols = [
        "gene_ID",
        "transcript_ID",
        "annot_gene_id",
        "annot_transcript_id",
        "annot_gene_name",
        "annot_transcript_name",
        "n_exons",
        "length",
        "gene_novelty",
        "transcript_novelty",
        "ISM_subtype",
    ]
    if df.columns.tolist()[:11] == cols:
        df = reformat_talon_abundance(df, how=how)

    # rename id ID column
    col = df.columns[0]
    if how == "gene":
        id_col = "gid"
    elif how == "iso":
        id_col = "tid"

    df.rename({col: id_col}, axis=1, inplace=True)

    # sum for gene level
    if how == "gene":
        df = df.groupby(id_col).sum().reset_index()

    # limit to just the transcripts already in the graph
    if how == "iso":
        sg_tids = sg.t_df.tid.tolist()
        ab_tids = df.tid.tolist()
        tids = list(set(sg_tids) & set(ab_tids))
        df = df.loc[df.tid.isin(tids)]

    # transpose to get adata format
    df.set_index(id_col, inplace=True)
    df = df.T

    # get adata components - obs, var, and X
    var = df.columns.to_frame()
    var.columns = [id_col]
    obs = df.index.to_frame()
    obs.columns = ["dataset"]
    X = sparse.csr_matrix(df.to_numpy())

    # create transcript-level adata object and filter out unexpressed transcripts
    adata = anndata.AnnData(var=var, obs=obs, X=X)
    genes, _ = sc.pp.filter_genes(adata, min_counts=1, inplace=False)
    adata = adata[:, genes]
    adata.layers["counts"] = adata.X

    return adata

In [3]:
def merge_adata_abundance(sg, adata, how="iso"):

    if how == "gene":
        dataset_list = sg.gene_datasets
        ab_bool = sg.has_gene_abundance()
        sg_adata = sg.gene_adata
    elif how == "iso":
        dataset_list = sg.datasets
        ab_bool = sg.has_abundance()
        sg_adata = sg.adata

    print(adata)

    # add each dataset to list of "datasets", check if any are already there!
    datasets = adata.obs.dataset.tolist()
    for d in datasets:
        if d in dataset_list:
            raise ValueError("Dataset {} already present in the SwanGraph.".format(d))
    dataset_list.extend(datasets)

    print()
    if len(datasets) <= 5:
        print(
            "Adding abundance for datasets {} to SwanGraph.".format(", ".join(datasets))
        )
    else:
        mini_datasets = datasets[:5]
        n = len(datasets) - len(mini_datasets)
        print(
            "Adding abundance for datasets {}... (and {} more) to SwanGraph".format(
                ", ".join(mini_datasets), n
            )
        )

    # if there is preexisting abundance data in the SwanGraph, concatenate
    # otherwise, adata is the new transcript level adata
    if not ab_bool:

        # create transcript-level adata object
        sg_adata = adata

        # add counts as layers
        sg_adata.layers["counts"] = sg_adata.X
        print("Calculating TPM...")
        sg_adata.layers["tpm"] = sparse.csr_matrix(
            calc_tpm(sg_adata, recalc=True).to_numpy()
        )

        if not sg.sc and how == "iso":
            print("Calculating PI...")
            sg_adata.layers["pi"] = sparse.csr_matrix(
                calc_pi(sg_adata, sg.t_df)[0].to_numpy()
            )
    else:

        # first set current layer to be counts
        sg_adata.X = sg_adata.layers["counts"]

        # concatenate existing adata with new one
        # outer join to add all new transcripts (that are from added
        # annotation or transcriptome) to the abundance
        uns = sg_adata.uns
        sg_adata = sg_adata.concatenate(adata, join="outer", index_unique=None)
        sg_adata.uns = uns

        # recalculate pi and tpm
        print("Calculating TPM...")
        sg_adata.layers["tpm"] = sparse.csr_matrix(
            calc_tpm(sg_adata, recalc=True).to_numpy()
        )

        if not sg.sc and how == "iso":
            print("Calculating PI...")
            sg_adata.layers["pi"] = sparse.csr_matrix(
                calc_pi(sg_adata, sg.t_df)[0].to_numpy()
            )

    # # add abundance for edges, TSS per gene, and TES per gene
    # if how == 'iso':
    #     print('Calculating edge usage...')
    #     sg.create_edge_adata()
    #     print('Calculating TSS usage...')
    #     sg.create_end_adata(kind='tss')
    #     print('Calculating TES usage...')
    #     sg.create_end_adata(kind='tes')

    # set abundance flag to true
    # and make adata object
    if how == "iso":
        sg.abundance = True
        sg.adata = sg_adata
    elif how == "gene":
        sg.gene_abundance = True
        sg.gene_adata = sg_adata

In [4]:
def create_end_adata(self, kind):
    """
    Create a tss / tes-level adata object. Enables calculating tss / tes
    usage across samples.

    Parameters:
        kind (str): Choose from 'tss' or 'tes'
    """

    # limit to only expresed transcripts
    t_df = sg.t_df.copy(deep=True)
    t_df = t_df.loc[sg.adata.var.index.tolist()]
    df = get_ends(t_df, kind)

    # get a mergeable transcript expression df
    tid = sg.adata.var.index.tolist()
    obs = sg.adata.obs.index.tolist()
    data = sg.adata.layers["counts"].transpose()
    t_exp_df = pd.DataFrame.sparse.from_spmatrix(columns=obs, data=data, index=tid)
    t_exp_df = t_exp_df.merge(t_df, how="left", left_index=True, right_index=True)

    # merge counts per transcript with end expression
    df = df.merge(t_exp_df, how="left", left_index=True, right_index=True)

    # sort based on vertex id
    df.sort_index(inplace=True, ascending=True)

    # set index to gene ID, gene name, and vertex id
    df.reset_index(drop=True, inplace=True)
    df.set_index(["gid", "gname", "vertex_id"], inplace=True)
    df = df[sg.datasets]

    # groupby on gene and assign each unique TSS / gene combo an ID
    # use dense representation b/c I've already removed 0 counts and sparse
    # gb operations are known to be slow in pandas
    # https://github.com/pandas-dev/pandas/issues/36123
    # maybe try this? :
    # https://cmdlinetips.com/2019/03/how-to-write-pandas-groupby-function-using-sparse-matrix/
    id_col = "{}_id".format(kind)
    name_col = "{}_name".format(kind)
    df = df.copy()
    df.reset_index(inplace=True)
    df[sg.datasets] = df[sg.datasets].sparse.to_dense()
    df = df.groupby(["gid", "gname", "vertex_id"]).sum().reset_index()

    df["end_gene_num"] = (
        df.sort_values(["gid", "vertex_id"], ascending=[True, True])
        .groupby(["gid"])
        .cumcount()
        + 1
    )
    df[id_col] = df["gid"] + "_" + df["end_gene_num"].astype(str)
    df[name_col] = df["gname"] + "_" + df["end_gene_num"].astype(str)
    df.drop("end_gene_num", axis=1, inplace=True)

    # obs, var, and X tables for new data
    var_cols = ["gid", "gname", "vertex_id", id_col, name_col]
    var = df[var_cols]
    var.set_index("{}_id".format(kind), inplace=True)
    df.drop(var_cols, axis=1, inplace=True)
    df = df[sg.adata.obs.index.tolist()]
    X = sparse.csr_matrix(df.transpose().values)
    obs = sg.adata.obs

    # create anndata
    adata = anndata.AnnData(var=var, obs=obs, X=X)

    # add counts and tpm as layers
    adata.layers["counts"] = adata.X
    adata.layers["tpm"] = sparse.csr_matrix(calc_tpm(adata, recalc=True).to_numpy())
    if not sg.sc:
        adata.layers["pi"] = sparse.csr_matrix(calc_pi(adata, adata.var)[0].to_numpy())

    # assign adata and clean up unstructured data if needed
    if kind == "tss":
        if sg.has_abundance():
            adata.uns = sg.tss_adata.uns
        sg.tss_adata = adata

    elif kind == "tes":
        if sg.has_abundance():
            adata.uns = sg.tss_adata.uns
        sg.tes_adata = adata

In [5]:
def add_abundance(sg, counts_file, how="iso"):
    adata = abundance_to_adata(sg, counts_file, how=how)
    merge_adata_abundance(sg, adata, how=how)

    return sg

In [11]:
s_pref = (
    "/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/swan_transcriptome"
)
annot = "/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/v29_cerberus.gtf"
ab = "/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/human_cerberus_abundance.tsv"
gene_ab = "/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/talon/human_talon_abundance.tsv"
gtf = "/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/cerberus.gtf"

In [13]:
sg = SwanGraph()
sg.add_annotation(annot)
sg.add_transcriptome(gtf)
sg.save_graph(s_pref)


Adding annotation to the SwanGraph

Adding transcriptome to the SwanGraph
Saving graph as /Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/swan_transcriptome.p


In [14]:
counts_file = "/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/human_cerberus_abundance.tsv"
how = "iso"

In [15]:
sg = add_abundance(sg, counts_file, how=how)

AnnData object with n_obs × n_vars = 138 × 157691
    obs: 'dataset'
    var: 'tid'
    layers: 'counts'

Adding abundance for datasets hepg2_1_1, imr90_1_1, mesenteric_fat_pad_1_1, ovary_1_1, hl60_1_1... (and 133 more) to SwanGraph
Calculating TPM...
Calculating PI...


/Users/fairliereese/Documents/programming/mortazavi_lab/bin/swan_vis/swan_vis/utils.py:410: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df.reset_index(inplace=True)


In [4]:
# def add_abundance(sg, counts_file, how='iso'):

In [16]:
sg = swan.read("/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/swan.p")

NameError: name 'swan' is not defined

In [ ]:
# add each dataset to list of "datasets", check if any are already there!
datasets = adata.obs.dataset.tolist()
for d in datasets:
    if d in sg.datasets:
        raise ValueError("Dataset {} already present in the SwanGraph.".format(d))
sg.datasets.extend(datasets)

print()
if len(datasets) <= 5:
    print("Adding abundance for datasets {} to SwanGraph.".format(", ".join(datasets)))
else:
    mini_datasets = datasets[:5]
    n = len(datasets) - len(mini_datasets)
    print(
        "Adding abundance for datasets {}... (and {} more) to SwanGraph".format(
            ", ".join(mini_datasets), n
        )
    )

# if there is preexisting abundance data in the SwanGraph, concatenate
# otherwise, adata is the new transcript level adata
if not sg.has_abundance():

    # create transcript-level adata object
    sg.adata = adata

    # add counts as layers
    sg.adata.layers["counts"] = sg.adata.X
    print("Calculating transcript TPM...")
    sg.adata.layers["tpm"] = sparse.csr_matrix(
        calc_tpm(sg.adata, recalc=True).to_numpy()
    )

    if not sg.sc:
        print("Calculating PI...")
        sg.adata.layers["pi"] = sparse.csr_matrix(
            calc_pi(sg.adata, sg.t_df)[0].to_numpy()
        )
else:

    # first set current layer to be counts
    sg.adata.X = sg.adata.layers["counts"]

    # concatenate existing adata with new one
    # outer join to add all new transcripts (that are from added
    # annotation or transcriptome) to the abundance
    uns = sg.adata.uns
    sg.adata = sg.adata.concatenate(adata, join="outer", index_unique=None)
    sg.adata.uns = uns

    # recalculate pi and tpm
    print("Calculating transcript TPM...")
    sg.adata.layers["tpm"] = sparse.csr_matrix(
        calc_tpm(sg.adata, recalc=True).to_numpy()
    )

    if not sg.sc:
        print("Calculating PI...")
        sg.adata.layers["pi"] = sparse.csr_matrix(
            calc_pi(sg.adata, sg.t_df)[0].to_numpy()
        )

# add abundance for edges, TSS per gene, and TES per gene
print("Calculating edge usage...")
sg.create_edge_adata()
print("Calculating TSS usage...")
sg.create_end_adata(kind="tss")
print("Calculating TES usage...")
sg.create_end_adata(kind="tes")

# set abundance flag to true
sg.abundance = True